In [15]:
#OLLAMAPool Server Prototype Code
import os
EndPoint_Queries=os.environ.get('EndPoint_Queries')
Endpoint_Results=os.environ.get('Endpoint_Results')
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_Queries is None:
    raise ValueError("EndPoint_Queries is not set")
if Endpoint_Results is None:
    raise ValueError("Endpoint_Results is not set")
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
print("Environment Variables are set OK")


Environment Variables are set OK


In [16]:
#Example Use case -> character description generator for fantasy RPG
#Takes a random dictionary of character traits and generates a written character description

races=["elf","human","dwarf","halfling","gnome"]
complections=["pale","dark","fair","ruddy","swarthy"]
heights=["tall","short","average"]
builds=["slender","stocky","muscular","portly"]
features=["sharp","soft","angular","round"]
eyes=["bright","dark","piercing","dull"]
hairs=["long","short","curly","straight","wavy"]
clothes=["fine","simple","elegant","gaudy","tattered"]
sophistication=["refined","crude","simple","complex"]
mannerisms=["graceful","clumsy","boisterous","quiet"]
virtues=["honest","loyal","brave","generous","kind"]


import random

#Come up with a random dictionary of character traits
def GenerateCharacterDescDictionary():
    traits={}
    race=random.choice(races)
    complection=random.choice(complections)
    height=random.choice(heights)
    build=random.choice(builds)
    feature=random.choice(features)
    eye=random.choice(eyes)
    hair=random.choice(hairs)
    clothe=random.choice(clothes)
    sophisticate=random.choice(sophistication)
    mannerism=random.choice(mannerisms)
    virtue=random.choice(virtues)
    traits={"race":race,"complection":complection,"height":height,"build":build,"feature":feature,"eye":eye,"hair":hair,"clothes":clothe,"sophistication":sophisticate,"mannerism":mannerism,"virtue":virtue}
    return traits

#Turn the dictionary into a line separated string
def GenerateCharacterDescList(traits):
    desc="They have the followng traits:\n"
    for key in traits:
        desc+=key+": "+traits[key]+"\n"
    return desc
    
            
    
#Test code
traits=GenerateCharacterDescDictionary()
print(GenerateCharacterDescList(traits))


They have the followng traits:
race: gnome
complection: dark
height: tall
build: muscular
feature: round
eye: dull
hair: wavy
clothes: gaudy
sophistication: refined
mannerism: quiet
virtue: honest



In [17]:
#Produce sample character descriptions
totalRecords=20
requestTexts=[]
for i in range(totalRecords):
    traits=GenerateCharacterDescDictionary()
    requestTexts.append(GenerateCharacterDescList(traits))
requestTexts

['They have the followng traits:\nrace: halfling\ncomplection: pale\nheight: short\nbuild: portly\nfeature: sharp\neye: bright\nhair: straight\nclothes: fine\nsophistication: crude\nmannerism: boisterous\nvirtue: loyal\n',
 'They have the followng traits:\nrace: dwarf\ncomplection: pale\nheight: average\nbuild: muscular\nfeature: sharp\neye: dull\nhair: straight\nclothes: simple\nsophistication: simple\nmannerism: quiet\nvirtue: loyal\n',
 'They have the followng traits:\nrace: dwarf\ncomplection: ruddy\nheight: short\nbuild: slender\nfeature: sharp\neye: dull\nhair: straight\nclothes: fine\nsophistication: refined\nmannerism: quiet\nvirtue: brave\n',
 'They have the followng traits:\nrace: elf\ncomplection: fair\nheight: average\nbuild: muscular\nfeature: soft\neye: dark\nhair: long\nclothes: fine\nsophistication: refined\nmannerism: graceful\nvirtue: kind\n',
 'They have the followng traits:\nrace: gnome\ncomplection: pale\nheight: short\nbuild: slender\nfeature: sharp\neye: piercing

In [18]:
#Busines Classes
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage

class LLMRequest:
    def __init__(self,Model:str="",systemMessage:str="",query:str=""):
        self.UUID=str(uuid.uuid4())
        self.Model=Model
        self.systemMessage=systemMessage
        self.query=query
        self.Messages=[{'role': 'system', 'content': systemMessage},
                       {'role': 'user', 'content': query}]

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMResult:
    def __init__(self,UUID:str="",result:str="",errorMsg:str="",timeDelta:str=""):
        self.UUID=UUID
        self.result=result
        self.errorMsg=errorMsg
        self.HasError=errorMsg!=""
        self.timeDelta=timeDelta

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMRequestResultPair():
    def __init__(self,request:LLMRequest,result:LLMResult=None):
        self.uuid=request.UUID
        self.request=request
        self.result=result


In [19]:
#Main OLLAMA Calling Function / Example code which posts data to queue
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage



#Post to a service bus queue    
def AzurePost_ServiceBus(connection_string: str, queue_name: str, json_payload):
    with ServiceBusClient.from_connection_string(connection_string) as client:
        sender = client.get_queue_sender(queue_name=queue_name)
        message = ServiceBusMessage(json.dumps(json_payload))
        with sender:
            sender.send_messages(message)
            print(f"Queued message: {json_payload['UUID']}")

#Take a list of queries and turn them into results
def BatchTransForm(Model:str,systemMessage:str,queries:List[str],DynamicUI:bool=True,WebUI:bool=False,ShowWebUI:bool=False)->List[str]:
    
    #Put every prompt into the LLM_Requests pool
    LLMrequests=[]
    for query in queries:
        request=LLMRequest(Model,systemMessage,query)
        LLMrequests.append(request)
        AzurePost_ServiceBus(EndPoint_Queries,"llm-queries",request.to_json())
    
    #Monitor the results queue
    return LLMrequests
    
    #Output the results
    
#Test code
systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""
#queuedRequests=BatchTransForm("mistral-small",systemPrompt,requestTexts)
queuedRequests=BatchTransForm("llama3.1",systemPrompt,requestTexts)



Queued message: 3c5a12f4-60e2-4691-be26-368e69248623
Queued message: 991dc77b-82be-4d63-b206-9abdde999286
Queued message: 0285c84d-a67a-4ef4-940d-19d0e8ca33d5
Queued message: 523597f7-89e9-422b-a8bf-78918fd3173d
Queued message: 62a33ddf-c851-4566-b23c-9e8e5bcc7cf9
Queued message: 8c8f9139-fd4d-41cb-9b24-cc607e3e66a5
Queued message: efdfd80b-8156-45ef-a50a-fccedbf873d5
Queued message: a4bcfa60-2817-4652-ba17-408a47b5b327
Queued message: d5950e18-ca9b-4a83-8d76-04212cba4ac6
Queued message: 4da1d3a0-a42c-4408-ac5e-6264f0137fbe
Queued message: 41645a39-5c3b-468b-9bc5-8783e3cae86d
Queued message: bf54740b-c13f-4304-99d0-6ead04c29b75
Queued message: 1c4f161f-32cb-499c-8c36-0cf7db86acf5
Queued message: ba84b41d-c4d4-4614-948b-96a910c1275d
Queued message: 9f63e280-2f42-4bc9-af5f-4fa292f71663
Queued message: 19d15ebb-6b47-4da2-89c7-390a5443addc
Queued message: 60eef63e-6366-4a5b-97db-b1c0e57ad33d
Queued message: e58fb894-b0a1-4730-a395-3c41e12a2436
Queued message: 230ff67c-59ba-4ce4-b6cb-9d34c5

In [20]:
#Main Result Handling Loop
import time
from typing import List
from azure.servicebus import ServiceBusClient, ServiceBusMessage

#Turn queud requests into dictionary
queueDict={}
for request in queuedRequests:
    queueDict[request.UUID]=LLMRequestResultPair(request)
totalResults=0
invalidResults=0

def ProcessResult(result:LLMResult):
    global totalResults
    global invalidResults
    if result.UUID in queueDict:
        print(f"Result UUID: {result.UUID} found in queueDict")
        queueDict[result.UUID].result=result
        totalResults+=1
    else:
        print(f"Result UUID: {result.UUID} not found in queueDict")
        invalidResults+=1

# Replace with your connection string and queue name
queue_name = "llm-responses"
servicebus_client = ServiceBusClient.from_connection_string(conn_str=Endpoint_Results)

def receive_messages_from_queue():
    # Create a receiver for the queue
    with servicebus_client.get_queue_receiver(queue_name=queue_name) as receiver:
        print("Receiving messages from the queue...")
        
        # Receive messages in a batch, you can specify max_message_count as needed
        received_msgs = receiver.receive_messages(max_message_count=10, max_wait_time=5)

        for msg in received_msgs:
            # Print the message payload
            print(f"Received message: {str(msg)}")
            receiver.complete_message(msg)
            
            llmRequest=LLMResult()
            llmRequest.from_json(str(msg))
            #llmRequest.from_json(json.loads(str(msg)))
            ProcessResult(llmRequest)
            
            # Accept the message to remove it from the queue
            #receiver.complete_message(msg)

# Call the function to read from the queue
while totalResults<len(queueDict):
    receive_messages_from_queue()
    print(f"Total Results: {totalResults}, Invalid Results: {invalidResults}")



Receiving messages from the queue...
Received message: {"UUID": "3c5a12f4-60e2-4691-be26-368e69248623", "result": {"model": "llama3.1", "created_at": "2024-09-26T12:07:38.362405815Z", "message": {"role": "assistant", "content": "Here's a character I came up with based on these traits:\n\nName: Bramble Burrowes\n\nBramble was a stout and jovial halfling, with a pale complexion that only seemed to glow in the warm light of a crackling fire. He stood at a mere two feet nine inches tall, but his portly build made him appear almost... substantial, as if he were a tiny island of prosperity amidst the rolling green hills of the countryside.\n\nHis features were sharp and well-defined, with high cheekbones and a pointed chin that seemed to be perpetually set in a scowl. But this was only a facade - Bramble's bright eyes sparkled with mirth and good humor, as if he were privy to some delightful joke that no one else quite understood.\n\nHis straight hair was a rich brown color, cut close to the

In [21]:
#Edge Case -> delibrately fail a few requests

systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""

#Invalid model!
#queuedRequests=BatchTransForm("herdepr28.1",systemPrompt,requestTexts[0:1])

